<a href="https://colab.research.google.com/github/wyklq/llmtune/blob/main/llmtune_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMTune: 4-Bit Finetuning of LLMs on a Consumer GPU
# Demo on Google Colab with T4 GPU (16G)
##1. Installation



### pip install -r requirements.txt (Note, pytorch is already installed)

In [ ]:
!pip install sentencepiece==0.1.97
!pip install git+https://github.com/huggingface/transformers@9417c924af539be5f941c8a709a96b60dfe29eb3#egg=transformers
!pip install git+https://github.com/huggingface/peft@ff9a1edbfd2d405b86d50a2e5299cc1bbd49d887#egg=peft
!pip install datasets==2.10.1